In [33]:
case class Article(
                    name: String,
                    priceWithoutCut: Double,
                    cutPercentage: Double,
                    category: String,
                    vatPercentage: Double,
                    quantity: Int,
                    ingredients: List[String]
                    ) {

  override def toString = this.name
  
}

defined class Article

In [34]:
object ShoppingList {

  def get = List(
    Article("Boeuf", 12.3, 0, "Viandes", 10, 1, List("Boeuf")),
    Article("Porc", 7.99, 5, "Viandes", 10, 1, List("Porc")),
    Article("Sauce Tomate", 2, 0, "Conserves", 10, 3, List("Tomate", "Eau", "Sel", "Sucre", "Conservateur")),
    Article("Petits Pois", 3.5, 10, "Conserves", 10, 2, List("Petits pois", "Eau", "Sel", "Conservateur")),
    Article("Thon", 1.50, 0, "Conserves", 20, 3, List("Thon", "Huile", "Sel", "Eau", "Conservateur"))
  )
}

defined object ShoppingList

In [35]:
val shoppingList = ShoppingList.get

shoppingList: List[Article] = List(Boeuf, Porc, Sauce Tomate, Petits Pois, Thon)

In [36]:
def articlesWithMeat:List[Article] = 
    shoppingList.filter(x => x.category == "Viandes")

println("Les articles de catégorie viande sont : " + articlesWithMeat)

Les articles de catégorie viande sont : List(Boeuf, Porc)


defined function articlesWithMeat

In [37]:
def pricesWithVat:List[(String, Double)] = 
    shoppingList.map( 
        x => (x.name,x.priceWithoutCut*(1+x.vatPercentage*0.01)))

println("Les prix TTC sont : " + pricesWithVat)

Les prix TTC sont : List((Boeuf,13.530000000000001), (Porc,8.789000000000001), (Sauce Tomate,2.2), (Petits Pois,3.8500000000000005), (Thon,1.7999999999999998))


defined function pricesWithVat

In [38]:
def ingredientsList:List[String] = 
    shoppingList.flatMap(x => x.ingredients).distinct

println("Les ingrédients sont : " + ingredientsList)

Les ingrédients sont : List(Boeuf, Porc, Tomate, Eau, Sel, Sucre, Conservateur, Petits pois, Thon, Huile)


defined function ingredientsList

In [39]:
val l = List(("Un",1),("Deux",2),("Trois",3),("Quatre",4))
l.map( e => e._2)

l: List[(String, Int)] = List(("Un", 1), ("Deux", 2), ("Trois", 3), ("Quatre", 4))
res38_1: List[Int] = List(1, 2, 3, 4)

In [40]:
def getArticleTotalPrice:(Article) => Double =
      a => a.priceWithoutCut*(1+0.01*a.vatPercentage)*(1-0.01*a.cutPercentage)*a.quantity

def totalInvoice:Double = 
    shoppingList
        .map(getArticleTotalPrice)
        .sum

println("Le total de la facture est : " + totalInvoice)

Le total de la facture est : 40.80955


defined function getArticleTotalPrice
defined function totalInvoice

In [42]:
def totalFor(category:String):Double =shoppingList
    .filter( x => x.category == category)
    .map(getArticleTotalPrice)
    .sum

println("Le total pour la catégorie viande est : " + totalFor("Viandes"))

Le total pour la catégorie viande est : 21.879550000000002


defined function totalFor

In [45]:
def sumByFirstTupleElement(acc:List[(String, Double)], element:(String, Double)) = (acc, element) match {
    case (Nil, a) => a::Nil
    case ((x1, x2)::xs, (a,b)) if x1 == a => (x1, x2 + b)::xs
    case (x::xs, a) => a::x::xs
  }

def priceByCategory:List[(String, Double)] = shoppingList
    .map(x => (x.category,getArticleTotalPrice(x)))
    .sortBy(_._1)
    .foldLeft(List[(String, Double)]())(sumByFirstTupleElement)

println("Les prix par catégorie sont : " + priceByCategory)

Les prix par catégorie sont : List((Viandes,21.879550000000002), (Conserves,18.93))


defined function sumByFirstTupleElement
defined function priceByCategory

In [47]:
def priceByCategory:List[(String, Double)] = shoppingList
  .groupBy(_.category)
  .mapValues(_.map(getArticleTotalPrice).sum)
  .toList

println("Les prix par catégorie sont : " + priceByCategory)

Les prix par catégorie sont : List((Viandes,21.879550000000002), (Conserves,18.93))


defined function priceByCategory

In [48]:
def priceByCategory:List[(String, Double)] = shoppingList
  .groupBy(_.category)
    .map{ x => 
        x match {
            case(key, values) => (key, values.map(getArticleTotalPrice).sum)
        } }
    .toList

println("Les prix par catégorie sont : " + priceByCategory)

Les prix par catégorie sont : List((Viandes,21.879550000000002), (Conserves,18.93))


defined function priceByCategory

In [51]:
def totalVat:Double = shoppingList
    .map(getArticleTotalVat)
    .reduce((prix1,prix2) => prix1 + prix2)

def getArticleTotalVat: (Article) => Double = {
    article => article.priceWithoutCut * article.vatPercentage / 100 * (1 - article.cutPercentage / 100) * article.quantity
  }

println("Le total de TVA est : " + totalVat)

Le total de TVA est : 4.11905


defined function totalVat
defined function getArticleTotalVat

In [50]:
def totalCut:Double = shoppingList
    .map(getArticleTotalCut)
    .foldLeft(0d)((prix1, prix2) => prix1 + prix2)

def getArticleTotalCut: (Article) => Double = {
    article => article.priceWithoutCut * (1 + article.vatPercentage / 100) * article.cutPercentage / 100 * article.quantity
  }


println("Le total des réduction est : " + totalCut)


Le total des réduction est : 1.2094500000000001


defined function totalCut
defined function getArticleTotalCut

In [49]:
def topIngredients(i:Int):List[String] = shoppingList
    .flatMap(x => x.ingredients)
    .map(x => (x, 1d))
    .sortBy(_._1)
    .foldLeft(List[(String, Double)]())(sumByFirstTupleElement)
    .sortBy(x => - x._2)
    .map(x => x._1)
    .take(3)

println("Les trois premiers ingrédients sont : " + topIngredients(3))

Les trois premiers ingrédients sont : List(Sel, Eau, Conservateur)


defined function topIngredients

In [13]:
def allArticleNames:String = ""

println("Le nom de tous les articles sont : " + allArticleNames)

Les articles de catégorie viande sont : List(Boeuf, Porc)
Les prix TTC sont : List((Boeuf,13.530000000000001), (Porc,8.789000000000001), (Sauce Tomate,2.2), (Petits Pois,3.8500000000000005), (Thon,1.7999999999999998))
Les ingrédients sont : List(Boeuf, Porc, Tomate, Eau, Sel, Sucre, Conservateur, Petits pois, Eau, Sel, Conservateur, Thon, Huile, Sel, Eau, Conservateur)
Le total de la facture est : 0.0
Le total pour la catégorie viande est : 0.0
Les prix par catégorie sont : List()
Le total de TVA est : 0.0
Le total des réduction est : 0.0
Les trois premiers ingrédients sont : List()
Le nom de tous les articles sont : 
